<a href="https://colab.research.google.com/github/beefburger-stock/-/blob/main/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install selenium pandas
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!apt-get install -y xvfb

!chromedriver --version
!pip install --upgrade selenium pandas chromedriver-autoinstaller

!pip install --upgrade selenium pandas chromedriver-autoinstaller
!apt-get update
!apt-get install -y chromium-browser xvfb
!pip install chromedriver-autoinstaller




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (113 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgr

In [10]:
import chromedriver_autoinstaller
import subprocess
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
from io import StringIO
import time
import requests

# 自動安裝適用的ChromeDriver
chromedriver_autoinstaller.install()

# 初始化Chrome選項
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 啟動虛擬顯示
subprocess.Popen(['Xvfb', ':1', '-screen', '0', '1024x768x16', '-ac'])

# 設置顯示環境變量
options.add_argument('--display=:1')

# 初始化ChromeDriver
driver = webdriver.Chrome(options=options)

# 獲取當前日期並格式化為適合URL的字符串
today = datetime.date.today()
date_str = today.strftime('%Y%m%d')  # 格式化為YYYYMMDD格式

# 上市盤後鉅額交易URL
twse_url = f"https://www.twse.com.tw/zh/trading/block/bfiauu.html?date={date_str}"

# 櫃買盤後鉅額交易URL
tpex_url = "https://www.tpex.org.tw/web/stock/block_trade/daily_qutoes/block_day.php?l=zh-tw"

def scrape_twse():
    driver.get(twse_url)

    # 等待頁面加載 (增加顯式等待)
    try:
        select_element = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[2]/main/div[2]/hgroup/div/div[1]/select"))
        )
        select_element.click()
        option_all = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[2]/main/div[2]/hgroup/div/div[1]/select/option[5]"))
        )
        option_all.click()

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div[2]/main/div[2]/div[2]"))
        )

        page_source = driver.page_source
        tables = pd.read_html(StringIO(page_source), converters={'證券代號': str})

        df = tables[0]
        df_filtered = df[['證券代號', '證券名稱', '成交價', '成交股數']]
        df_filtered = df_filtered[~df_filtered['證券代號'].str.startswith('00')]
        exclude_list = ['2330', '2308', '2002', '2454', '2881', '2886', '2891', '3045', '3711', '6669', '1590', '2317', '2412', '2603', '6409', '3037', '3017', '6147', '6643', '5009']
        df_filtered = df_filtered[~df_filtered['證券代號'].isin(exclude_list)]
        df_filtered = df_filtered[df_filtered['證券代號'] != '總計']
        df_filtered.reset_index(drop=True, inplace=True)

        return "上市盤後鉅額交易\n" + df_filtered.to_string(index=False)

    except Exception as e:
        print("未找到表格，可能原因：", e)
        return None

def scrape_tpex():
    driver.get(tpex_url)

    try:
        first_page_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/center/div[3]/div[2]/div[4]/div/div[3]/a[1]"))
        )
        first_page_button.click()
    except Exception as e:
        print("未找到“第一頁”按鈕，可能原因：", e)
        driver.quit()

    def scrape_current_page():
        try:
            table_element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, "//table"))
            )
            page_source = driver.page_source
            tables = pd.read_html(StringIO(page_source), converters={'代號': str})
            return tables[0]
        except Exception as e:
            print("未找到表格，可能原因：", e)
            return None

    all_data = []
    while True:
        df = scrape_current_page()
        if df is not None:
            df_filtered = df[['代號', '名稱', '成交價格(元)', '成交股數']]
            df_filtered = df_filtered[~df_filtered['代號'].str.startswith('00')]
            exclude_list = ['8069', '3264', '3324', '5347', '3105', '6147', '6643', '5009']
            df_filtered = df_filtered[~df_filtered['代號'].isin(exclude_list)]
            all_data.append(df_filtered)

        try:
            next_page_button = driver.find_element(By.XPATH, "/html/body/center/div[3]/div[2]/div[4]/div/div[3]/a[3]")
            next_page_class = next_page_button.get_attribute('class')
            if 'disable' in next_page_class:
                break
            else:
                next_page_button.click()
                time.sleep(2)
        except Exception as e:
            print("未找到“下一頁”按鈕，可能原因：", e)
            break

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.reset_index(drop=True, inplace=True)

        return "櫃買盤後鉅額交易\n" + final_df.to_string(index=False)

    else:
        print("未找到任何表格")
        return None

def send_line_notify(message, tokens):
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    for token in tokens:
        headers['Authorization'] = f'Bearer {token}'
        data = {
            'message': message
        }
        response = requests.post(line_notify_api, headers=headers, data=data)
        print(f'Status Code for token {token}:', response.status_code)  # 打印回應狀態碼以確保訊息已發送

try:
    twse_data = scrape_twse()
    tpex_data = scrape_tpex()
    if twse_data:
        print(twse_data)
    if tpex_data:
        print(tpex_data)

    message = ""
    if twse_data:
        message += twse_data + "\n"
    if tpex_data:
        message += tpex_data + "\n"

    if message:
        line_notify_tokens = [
            'qxnkqbkJ2E78E8hk08c0qWCj4kiApg3viBHjBbWAf7S',
            'JEjZ1Kb4mnarduX57LVo6W3xUyxArUEiKYIGY5meYny'
        ]
        send_line_notify(message, line_notify_tokens)

finally:
    driver.quit()

上市盤後鉅額交易
證券代號 證券名稱  成交價  成交股數
    5007     三星    55.0    300000
櫃買盤後鉅額交易
代號   名稱  成交價格(元)  成交股數
4105   東洋         83.20     26000
4107   邦特        130.50     43000
4111   濟生         31.75     65000
4114   健喬         44.75     48000
4128   中天         40.20     80000
4147   中裕         88.90     36000
4167 松瑞藥         23.35      1000
5530   龍巖         56.50     18000
5536  聖暉*        242.00     21000
5603   陸海         28.50     78000
6261   久元         81.90     18000
8354   冠好         19.15     28000
8383   千附         47.15     19000
Status Code for token qxnkqbkJ2E78E8hk08c0qWCj4kiApg3viBHjBbWAf7S: 200
Status Code for token JEjZ1Kb4mnarduX57LVo6W3xUyxArUEiKYIGY5meYny: 200


In [11]:
!pip install pyinstaller

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.2/683.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 35.5 MB/s eta 0:00:00
